# 策略分析示例 | Strategy Analysis Example

本笔记本演示如何使用工具包进行策略分析

This notebook demonstrates how to use the toolkit for strategy analysis

In [ ]:
# 导入必要的库 | Import necessary libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils.data_loader import load_stock_data
from utils.indicators import calculate_all_indicators, calculate_ma
from utils.performance import (
    calculate_returns, 
    generate_performance_report, 
    print_performance_report
)

# 设置绘图风格 | Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. 数据加载 | Data Loading

In [ ]:
# 加载股票数据 | Load stock data
stock_code = '000001.XSHE'  # 平安银行 | Ping An Bank
start_date = '2023-01-01'
end_date = '2023-12-31'

data = load_stock_data(stock_code, start_date, end_date)
print(f"数据形状 | Data shape: {data.shape}")
data.head()

## 2. 技术指标计算 | Technical Indicators

In [ ]:
# 计算所有技术指标 | Calculate all technical indicators
data_with_indicators = calculate_all_indicators(data)

# 显示包含指标的数据 | Display data with indicators
print("可用指标 | Available indicators:")
print(data_with_indicators.columns.tolist())

data_with_indicators.tail()

## 3. 价格与均线可视化 | Price and Moving Average Visualization

In [ ]:
# 绘制价格和均线 | Plot price and moving averages
plt.figure(figsize=(14, 7))

plt.plot(data_with_indicators.index, data_with_indicators['close'], 
         label='收盘价 | Close Price', linewidth=2, alpha=0.8)
plt.plot(data_with_indicators.index, data_with_indicators['MA20'], 
         label='MA20', linewidth=1.5, alpha=0.7)
plt.plot(data_with_indicators.index, data_with_indicators['MA60'], 
         label='MA60', linewidth=1.5, alpha=0.7)

plt.title(f'{stock_code} 价格走势与均线 | Price Trend with Moving Averages', fontsize=16)
plt.xlabel('日期 | Date', fontsize=12)
plt.ylabel('价格 | Price', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 布林带分析 | Bollinger Bands Analysis

In [ ]:
# 绘制布林带 | Plot Bollinger Bands
plt.figure(figsize=(14, 7))

plt.plot(data_with_indicators.index, data_with_indicators['close'], 
         label='收盘价 | Close Price', linewidth=2, color='blue', alpha=0.8)
plt.plot(data_with_indicators.index, data_with_indicators['BB_upper'], 
         label='上轨 | Upper Band', linewidth=1, linestyle='--', color='red', alpha=0.6)
plt.plot(data_with_indicators.index, data_with_indicators['BB_middle'], 
         label='中轨 | Middle Band', linewidth=1, linestyle='--', color='gray', alpha=0.6)
plt.plot(data_with_indicators.index, data_with_indicators['BB_lower'], 
         label='下轨 | Lower Band', linewidth=1, linestyle='--', color='green', alpha=0.6)

# 填充布林带区域 | Fill Bollinger Bands area
plt.fill_between(data_with_indicators.index, 
                 data_with_indicators['BB_upper'], 
                 data_with_indicators['BB_lower'], 
                 alpha=0.1, color='gray')

plt.title('布林带分析 | Bollinger Bands Analysis', fontsize=16)
plt.xlabel('日期 | Date', fontsize=12)
plt.ylabel('价格 | Price', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. MACD指标分析 | MACD Analysis

In [ ]:
# 绘制MACD | Plot MACD
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# 上图：价格 | Top plot: Price
ax1.plot(data_with_indicators.index, data_with_indicators['close'], 
         label='收盘价 | Close Price', linewidth=2)
ax1.set_ylabel('价格 | Price', fontsize=12)
ax1.set_title('价格与MACD指标 | Price and MACD Indicator', fontsize=16)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 下图：MACD | Bottom plot: MACD
ax2.plot(data_with_indicators.index, data_with_indicators['MACD'], 
         label='MACD', linewidth=1.5)
ax2.plot(data_with_indicators.index, data_with_indicators['MACD_signal'], 
         label='信号线 | Signal', linewidth=1.5)
ax2.bar(data_with_indicators.index, data_with_indicators['MACD_hist'], 
        label='柱状图 | Histogram', alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_xlabel('日期 | Date', fontsize=12)
ax2.set_ylabel('MACD', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. RSI指标分析 | RSI Analysis

In [ ]:
# 绘制RSI | Plot RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# 上图：价格 | Top plot: Price
ax1.plot(data_with_indicators.index, data_with_indicators['close'], 
         label='收盘价 | Close Price', linewidth=2)
ax1.set_ylabel('价格 | Price', fontsize=12)
ax1.set_title('价格与RSI指标 | Price and RSI Indicator', fontsize=16)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 下图：RSI | Bottom plot: RSI
ax2.plot(data_with_indicators.index, data_with_indicators['RSI'], 
         label='RSI', linewidth=2, color='purple')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, label='超买 | Overbought (70)')
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, label='超卖 | Oversold (30)')
ax2.axhline(y=50, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
ax2.fill_between(data_with_indicators.index, 70, 100, alpha=0.1, color='red')
ax2.fill_between(data_with_indicators.index, 0, 30, alpha=0.1, color='green')
ax2.set_xlabel('日期 | Date', fontsize=12)
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 性能分析 | Performance Analysis

In [ ]:
# 计算收益率 | Calculate returns
returns = calculate_returns(data)

# 生成性能报告 | Generate performance report
report = generate_performance_report(returns, initial_capital=100000)

# 打印报告 | Print report
print_performance_report(report)

## 8. 收益分布分析 | Returns Distribution Analysis

In [ ]:
# 绘制收益率分布 | Plot returns distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：收益率直方图 | Left: Returns histogram
axes[0].hist(returns.dropna() * 100, bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('日收益率 (%) | Daily Returns (%)', fontsize=12)
axes[0].set_ylabel('频数 | Frequency', fontsize=12)
axes[0].set_title('收益率分布 | Returns Distribution', fontsize=14)
axes[0].grid(True, alpha=0.3)

# 右图：累计收益 | Right: Cumulative returns
cumulative_returns = (1 + returns).cumprod() - 1
axes[1].plot(cumulative_returns.index, cumulative_returns * 100, linewidth=2)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1].set_xlabel('日期 | Date', fontsize=12)
axes[1].set_ylabel('累计收益率 (%) | Cumulative Returns (%)', fontsize=12)
axes[1].set_title('累计收益曲线 | Cumulative Returns', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. 总结 | Summary

本笔记本演示了：

1. 如何加载和处理股票数据
2. 如何计算常用技术指标
3. 如何可视化价格和指标
4. 如何进行性能分析

This notebook demonstrated:

1. How to load and process stock data
2. How to calculate common technical indicators
3. How to visualize prices and indicators
4. How to perform performance analysis